In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2023-10-19 23:12:45.154466: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-19 23:12:45.203077: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-19 23:12:45.212455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 23:12:46.049288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [3]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [4]:
results = read_csv_files_from_folder('results/')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

In [6]:
display(results)

,Unnamed: 0,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,28,Bank Marketing,Prejudice Remover,mcc_parity,0.451814,0.902919,0.493627,0.055961,0.041813,0.098765,0.493627,0.041813,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,33,Bank Marketing,Prejudice Remover,mcc_odds,0.469590,0.900295,0.498464,0.028874,0.075035,0.023956,0.498464,0.028874,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,38,Bank Marketing,Prejudice Remover,mcc_opportunity,0.286930,0.900787,0.493162,0.107477,0.008737,0.206232,0.493162,0.206232,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,43,Bank Marketing,Prejudice Remover,acc_parity,0.818690,0.901115,0.478296,0.113904,0.082425,0.134470,0.901115,0.082425,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,48,Bank Marketing,Prejudice Remover,acc_odds,0.879858,0.896851,0.480816,0.016993,0.057326,0.011917,0.896851,0.016993,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,342,German Credit,Fair Transition Loss,mcc_odds,0.172475,0.705000,0.354848,0.182373,0.153674,0.106413,0.354848,0.182373,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
1835,346,German Credit,Fair Transition Loss,mcc_opportunity,0.331920,0.745000,0.460063,0.115175,0.129345,0.128143,0.460063,0.128143,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
1836,350,German Credit,Fair Transition Loss,acc_parity,-0.096667,0.570000,-0.070943,0.632353,0.666667,0.764706,0.570000,0.666667,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
1837,354,German Credit,Fair Transition Loss,acc_odds,0.624027,0.770000,0.268941,0.145973,0.115937,0.056651,0.770000,0.145973,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)


In [7]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [8]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [9]:
methods = ['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']

In [10]:
if os.path.exists('multi_aso_data_list.json'):
    with open('multi_aso_data_list.json') as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open('multi_aso_data_list.json', 'w') as file:
        json.dump(multi_aso_data_list, file)

Model comparisons: 100%|█████████▉| 9990/10000 [00:51<00:00, 195.44it/s]

Model comparisons:   0%|          | 0/10000 [00:00<?, ?it/s]

Model comparisons:   0%|          | 19/10000 [00:00<00:52, 189.92it/s]

Model comparisons:   0%|          | 39/10000 [00:00<00:51, 194.17it/s]

Model comparisons:   1%|          | 59/10000 [00:00<00:50, 195.97it/s]

Model comparisons:   1%|          | 79/10000 [00:00<00:50, 197.18it/s]

Model comparisons:   1%|          | 99/10000 [00:00<00:50, 195.50it/s]

Model comparisons:   1%|          | 119/10000 [00:00<00:50, 194.98it/s]

Model comparisons:   1%|▏         | 139/10000 [00:00<00:50, 194.78it/s]

Model comparisons:   2%|▏         | 159/10000 [00:00<00:50, 195.31it/s]

Model comparisons:   2%|▏         | 179/10000 [00:00<00:50, 194.33it/s]

Model comparisons:   2%|▏         | 199/10000 [00:01<00:50, 194.54it/s]

Model comparisons:   2%|▏         | 219/10000 [00:01<00:50, 193.24it/s]

Model comparisons:   2%|▏         | 239/10000 [00:01<00:50, 193.23

In [11]:
aso_df_resume = []
reverse_aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df = aso_df[aso_df['method'] == 'Fair Transition Loss' ]
    aso_df = aso_df.drop(['Fair Transition Loss'], axis=1)
    aso_df = aso_df.drop(['method'], axis=1)
    aso_df_resume.append(aso_df)

    reverse_aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods).transpose()
    mapping = dict()
    for i, m in enumerate(methods):
        mapping[reverse_aso_df.columns[i]] = m + ' (reverse)'
    reverse_aso_df = reverse_aso_df.rename(columns=mapping)
    reverse_aso_df['method'] = methods
    reverse_aso_df['dataset'] = dataset
    reverse_aso_df['fitness_rule'] = fitness_rule
    reverse_aso_df = reverse_aso_df[reverse_aso_df['method'] == 'Fair Transition Loss' ]
    reverse_aso_df = reverse_aso_df.drop(['Fair Transition Loss (reverse)'], axis=1)
    reverse_aso_df = reverse_aso_df.drop(['method'], axis=1)
    reverse_aso_df_resume.append(reverse_aso_df)

print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'])
display(significance)
formatted_significance = significance.applymap(lambda x: '\\textbf{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )
formatted_significance.to_latex('tables/significance_resume.tex')

reverse_significance = pd.concat(reverse_aso_df_resume)
reverse_significance = reverse_significance.set_index(['fitness_rule', 'dataset'])
reverse_significance = reverse_significance.sort_values(by=['fitness_rule', 'dataset'])
reverse_formatted_significance = reverse_significance.applymap(lambda x: '\\textit{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )
reverse_formatted_significance.to_latex('tables/reverse_significance_resume.tex')

Significance Testing


Standard MLP (baseline)  \
fitness_rule    dataset                                      
acc_odds        Adult Income                      0.009382   
                Bank Marketing                    0.759610   
                Compas Recidivism                 0.006599   
                German Credit                     1.000000   
acc_opportunity Adult Income                      0.899651   
                Bank Marketing                    1.000000   
                Compas Recidivism                 0.009009   
                German Credit                     1.000000   
acc_parity      Adult Income                      0.006254   
                Bank Marketing                    0.258596   
                Compas Recidivism                 0.002068   
                German Credit                     1.000000   
mcc_odds        Adult Income                      0.039891   
                Bank Marketing                    0.454311   
                Compas Recidivism                 0.005326   
                German Credit                     1.000000   
mcc_opportunity Adult Income                      0.008385   
                Bank Marketing                    0.813081   
                Compas Recidivism                 0.004842   
                German Credit                     1.000000   
mcc_parity      Adult Income                      0.004548   
                Bank Marketing                    0.007935   
                Compas Recidivism                 0.005150   
                German Credit                     0.300344   

                                   Adversarial Debiasing  Prejudice Remover  \
fitness_rule    dataset                                                       
acc_odds        Adult Income                    0.231199           0.190276   
                Bank Marketing                  0.399582           0.824730   
                Compas Recidivism               0.449340           0.000000   
                German Credit                   0.121936           1.000000   
acc_opportunity Adult Income                    0.970868           1.000000   
                Bank Marketing                  0.382120           0.822045   
                Compas Recidivism               0.794141           0.000000   
                German Credit                   0.642496           1.000000   
acc_parity      Adult Income                    0.296407           0.288967   
                Bank Marketing                  1.000000           1.000000   
                Compas Recidivism               1.000000           0.107308   
                German Credit                   0.266664           1.000000   
mcc_odds        Adult Income                    0.255806           0.402994   
                Bank Marketing                  0.195346           0.118342   
                Compas Recidivism               0.570140           0.000000   
                German Credit                   0.117264           1.000000   
mcc_opportunity Adult Income                    0.011001           0.055793   
                Bank Marketing                  0.166773           0.244488   
                Compas Recidivism               1.000000           0.000000   
                German Credit                   0.215107           0.850813   
mcc_parity      Adult Income                    0.156742           1.000000   
                Bank Marketing                  0.000000           0.000000   
                Compas Recidivism               0.208813           0.000000   
                German Credit                   0.286074           0.423609   

                                   Gerry Fair Classifier  
fitness_rule    dataset                                   
acc_odds        Adult Income                    0.000000  
                Bank Marketing                  1.000000  
                Compas Recidivism               0.021341  
                German Credit                   1.000000  
acc_opportuni

In [12]:
for col, rev_col in zip(formatted_significance.columns, reverse_formatted_significance.columns):
    formatted_significance[col] = formatted_significance[col] + (' (' + reverse_formatted_significance[rev_col] + ')')
formatted_significance.to_latex('tables/combined_significance_resume.tex')
formatted_significance

Standard MLP (baseline)  \
fitness_rule    dataset                                     
acc_odds        Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing                0.76 (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit        1.00 (\textit{0.40})   
acc_opportunity Adult Income                  0.90 (1.00)   
                Bank Marketing                1.00 (0.78)   
                Compas Recidivism    \textbf{0.01} (0.99)   
                German Credit                 1.00 (0.59)   
acc_parity      Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing       \textbf{0.26} (1.00)   
                Compas Recidivism    \textbf{0.00} (1.00)   
                German Credit                 1.00 (0.75)   
mcc_odds        Adult Income         \textbf{0.04} (1.00)   
                Bank Marketing       \textbf{0.45} (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit                 1.00 (1.00)   
mcc_opportunity Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing                0.81 (1.00)   
                Compas Recidivism    \textbf{0.00} (1.00)   
                German Credit                 1.00 (0.84)   
mcc_parity      Adult Income         \textbf{0.00} (1.00)   
                Bank Marketing       \textbf{0.01} (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit        \textbf{0.30} (1.00)   

                                  Adversarial Debiasing     Prejudice Remover  \
fitness_rule    dataset                                                         
acc_odds        Adult Income       \textbf{0.23} (1.00)  \textbf{0.19} (1.00)   
                Bank Marketing     \textbf{0.40} (1.00)           0.82 (1.00)   
                Compas Recidivism  \textbf{0.45} (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.12} (1.00)  1.00 (\textit{0.16})   
acc_opportunity Adult Income                0.97 (1.00)  1.00 (\textit{0.49})   
                Bank Marketing     \textbf{0.38} (1.00)           0.82 (1.00)   
                Compas Recidivism           0.79 (1.00)  \textbf{0.00} (1.00)   
                German Credit               0.64 (1.00)           1.00 (0.82)   
acc_parity      Adult Income       \textbf{0.30} (1.00)  \textbf{0.29} (1.00)   
                Bank Marketing              1.00 (0.55)           1.00 (0.75)   
                Compas Recidivism           1.00 (0.51)  \textbf{0.11} (1.00)   
                German Credit      \textbf{0.27} (1.00)  1.00 (\textit{0.38})   
mcc_odds        Adult Income       \textbf{0.26} (1.00)  \textbf{0.40} (1.00)   
                Bank Marketing     \textbf{0.20} (1.00)  \textbf{0.12} (1.00)   
                Compas Recidivism           0.57 (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.12} (1.00)           1.00 (0.91)   
mcc_opportunity Adult Income       \textbf{0.01} (1.00)  \textbf{0.06} (1.00)   
                Bank Marketing     \textbf{0.17} (1.00)  \textbf{0.24} (1.00)   
                Compas Recidivism  1.00 (\textit{0.19})  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.22} (1.00)           0.85 (1.00)   
mcc_parity      Adult Income       \textbf{0.16} (1.00)           1.00 (1.00)   
                Bank Marketing     \textbf{0.00} (1.00)  \textbf{0.00} (1.00)   
                Compas Recidivism  \textbf{0.21} (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.29} (1.00)  \textbf{0.42} (1.00)   

                                  Gerry Fair Classifier  
fitness_rule    dataset                                  
acc_odds        Adult Income       \textbf{0.00} (1.00)  
                Bank Marketing              1.00 (0.80)  
                Compas Recidivism  \textbf{0.02} (1.00)  
                German Credit      1.00 (\textit{0

In [13]:
grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
grouped_results

fitness  \
                                                                   mean   
Fitness Rule Abvr       dataset       method                              
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.809534   
                                      Fair Transition Loss     0.787013   
                                      Adversarial Debiasing    0.756054   
                                      Standard MLP (baseline)  0.752260   
                                      Gerry Fair Classifier    0.704668   
...                                                                 ...   
Max(MCC - Stat. Parity) German Credit Fair Transition Loss     0.271735   
                                      Prejudice Remover        0.233727   
                                      Standard MLP (baseline)  0.223085   
                                      Gerry Fair Classifier    0.220550   
                                      Adversarial Debiasing    0.200070   

                                                                         \
                                                                    std   
Fitness Rule Abvr       dataset       method                              
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.022701   
                                      Fair Transition Loss     0.083934   
                                      Adversarial Debiasing    0.028345   
                                      Standard MLP (baseline)  0.041790   
                                      Gerry Fair Classifier    0.074245   
...                                                                 ...   
Max(MCC - Stat. Parity) German Credit Fair Transition Loss     0.082350   
                                      Prejudice Remover        0.087339   
                                      Standard MLP (baseline)  0.095145   
                                      Gerry Fair Classifier    0.088103   
                                      Adversarial Debiasing    0.172675   

                                                              Performance  \
                                                                     mean   
Fitness Rule Abvr       dataset       method                                
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover          0.845788   
                                      Fair Transition Loss       0.825826   
                                      Adversarial Debiasing      0.847730   
                                      Standard MLP (baseline)    0.849309   
                                      Gerry Fair Classifier      0.750809   
...                                                                   ...   
Max(MCC - Stat. Parity) German Credit Fair Transition Loss       0.354444   
                                      Prejudice Remover          0.328818   
                                      Standard MLP (baseline)    0.329618   
                                      Gerry Fair Classifier      0.291429   
                                      Adversarial Debiasing      0.367673   

                                                                         \
                                                                    std   
Fitness Rule Abvr       dataset       method                              
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.003691   
                                      Fair Transition Loss     0.069484   
                                      Adversarial Debiasing    0.002904   
                                      Standard MLP (baseline)  0.003078   
                                      Gerry Fair Classifier    0.085845   
...                                                                 ...   
Max(MCC - Stat. Parity) German Credit Fair Transition Loss     0.068050   
                                      Prejudice Remover        0.051833   
                                      Standard MLP (baseli

In [14]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
grouped_results[selected_columns].to_latex('tables/grouped_results.tex')